## 1. Configuration et importation des bibliothèques

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
from datetime import datetime
import json

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.width', 1000)

input_dir = "data/raw"
output_dir = "data/processed"
os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

db_path = os.path.join(output_dir, "pandemic_data.db")

print(f"Répertoire d'entrée: {input_dir}")
print(f"Répertoire de sortie: {output_dir}")
print(f"Base de données: {db_path}")

## 2. Extraction des données

In [ ]:
def extract_csv(file_path):
    """Extrait les données d'un fichier CSV"""
    try:
        df = pd.read_csv(file_path)
        print(f"Extraction réussie: {file_path}, {len(df)} lignes")
        return df
    except Exception as e:
        print(f"Erreur lors de l'extraction de {file_path}: {e}")
        return pd.DataFrame()

# Liste des fichiers à extraire
files_to_extract = [
    os.path.join(input_dir, "covid_19_clean_complete.csv"),
    os.path.join(input_dir, "worldometer_coronavirus_daily_data.csv"),
    os.path.join(input_dir, "owid-monkeypox-data.csv")
]

# Extraction des données
raw_data = {}
for file_path in files_to_extract:
    if os.path.exists(file_path):
        file_name = os.path.basename(file_path)
        raw_data[file_name] = extract_csv(file_path)
    else:
        print(f"Fichier non trouvé: {file_path}")

# Affichage des résultats
print(f"\nNombre de fichiers extraits: {len(raw_data)}")
for file_name, df in raw_data.items():
    if not df.empty:
        print(f"  - {file_name}: {len(df)} lignes, {len(df.columns)} colonnes")
        print(f"    Colonnes: {', '.join(df.columns)}")
        print(f"    Aperçu:")
        display(df.head(3))

## 3. Transformation des données

In [ ]:
def transform_covid_data(df, file_name):
    """Transforme les données COVID-19"""
    df_transformed = df.copy()
    
    # Transformation spécifique selon le fichier
    if "covid_19_clean_complete" in file_name:
        # Conversion des dates
        df_transformed['Date'] = pd.to_datetime(df_transformed['Date'])
        
        # Remplacement des valeurs manquantes
        for col in ['Confirmed', 'Deaths', 'Recovered', 'Active']:
            df_transformed[col] = df_transformed[col].fillna(0).astype(int)
        
        # Agrégation par pays et date
        df_transformed = df_transformed.groupby(['Country/Region', 'Date']).agg({
            'Confirmed': 'sum',
            'Deaths': 'sum',
            'Recovered': 'sum',
            'Active': 'sum'
        }).reset_index()
        
    elif "worldometer_coronavirus" in file_name:
        # Conversion des dates
        df_transformed['date'] = pd.to_datetime(df_transformed['date'])
        
        # Renommage des colonnes
        df_transformed = df_transformed.rename(columns={
            'country': 'Country/Region',
            'date': 'Date',
            'cumulative_total_cases': 'Confirmed',
            'cumulative_total_deaths': 'Deaths'
        })
        
        # Ajout des colonnes manquantes
        df_transformed['Recovered'] = 0
        df_transformed['Active'] = df_transformed['Confirmed'] - df_transformed['Deaths']
    
    print(f"Transformation de {file_name}: {len(df_transformed)} lignes")
    return df_transformed

def transform_monkeypox_data(df, file_name):
    """Transforme les données Monkeypox"""
    df_transformed = df.copy()
    
    # Conversion des dates
    df_transformed['date'] = pd.to_datetime(df_transformed['date'])
    
    # Renommage des colonnes
    df_transformed = df_transformed.rename(columns={
        'location': 'Country/Region',
        'date': 'Date',
        'total_cases': 'Confirmed',
        'total_deaths': 'Deaths'
    })
    
    # Remplacement des valeurs manquantes
    for col in ['Confirmed', 'Deaths']:
        if col in df_transformed.columns:
            df_transformed[col] = df_transformed[col].fillna(0).astype(int)
    
    # Ajout des colonnes manquantes
    df_transformed['Recovered'] = 0
    df_transformed['Active'] = df_transformed['Confirmed'] - df_transformed['Deaths']
    
    print(f"Transformation de {file_name}: {len(df_transformed)} lignes")
    return df_transformed

# Transformation des données
transformed_data = {}
for file_name, df in raw_data.items():
    if df.empty:
        continue
        
    if "monkeypox" in file_name.lower():
        transformed_data[file_name] = transform_monkeypox_data(df, file_name)
    else:
        transformed_data[file_name] = transform_covid_data(df, file_name)

# Affichage des résultats
print(f"\nNombre de fichiers transformés: {len(transformed_data)}")
for file_name, df in transformed_data.items():
    print(f"\nAperçu de {file_name} transformé ({len(df)} lignes):")
    display(df.head(3))

## 4. Préparation du schéma

In [ ]:
# Préparation de la table calendar
def prepare_calendar_table(transformed_data):
    """Prépare la table calendar"""
    all_dates = []
    for df in transformed_data.values():
        if 'Date' in df.columns:
            all_dates.extend(df['Date'].dt.strftime('%Y%m%d').astype(int).unique())
    
    unique_dates = sorted(set(all_dates))
    df_calendar = pd.DataFrame({
        'id': range(1, len(unique_dates) + 1),
        'date_value': unique_dates
    })
    
    print(f"Table calendar préparée: {len(df_calendar)} lignes")
    return df_calendar

# Préparation de la table location
def prepare_location_table(transformed_data):
    """Prépare la table location"""
    all_countries = []
    for df in transformed_data.values():
        if 'Country/Region' in df.columns:
            all_countries.extend(df['Country/Region'].unique())
    
    unique_countries = sorted(set(all_countries))
    
    # Mapping simplifié des continents
    continent_mapping = {
        'US': 'North America',
        'Canada': 'North America',
        'Brazil': 'South America',
        'France': 'Europe',
        'Germany': 'Europe',
        'China': 'Asia',
        'India': 'Asia',
        'Australia': 'Oceania',
        'South Africa': 'Africa'
    }
    
    df_location = pd.DataFrame({
        'id': range(1, len(unique_countries) + 1),
        'country': unique_countries,
        'continent': [continent_mapping.get(country, 'Unknown') for country in unique_countries]
    })
    
    print(f"Table location préparée: {len(df_location)} lignes")
    return df_location

# Préparation de la table pandemie
def prepare_pandemie_table():
    """Prépare la table pandemie"""
    df_pandemie = pd.DataFrame({
        'id': [1, 2],
        'type': ['COVID-19', 'Monkeypox']
    })
    
    print(f"Table pandemie préparée: {len(df_pandemie)} lignes")
    return df_pandemie

# Préparation de la table data
def prepare_data_table(transformed_data, df_calendar, df_location):
    """Prépare la table data"""
    # Création des dictionnaires pour les lookups
    date_to_id = dict(zip(df_calendar['date_value'], df_calendar['id']))
    country_to_id = dict(zip(df_location['country'], df_location['id']))
    
    # Préparation des données
    data_rows = []
    id_counter = 1
    
    for file_name, df in transformed_data.items():
        # Détermination du type de pandémie
        pandemie_id = 1  # COVID-19 par défaut
        if 'monkeypox' in file_name.lower():
            pandemie_id = 2  # Monkeypox
        
        # Traitement des données
        for _, row in df.iterrows():
            # Conversion de la date au format YYYYMMDD
            date_value = int(row['Date'].strftime('%Y%m%d'))
            
            # Récupération des IDs
            calendar_id = date_to_id.get(date_value)
            location_id = country_to_id.get(row['Country/Region'])
            
            if calendar_id is None or location_id is None:
                continue
            
            # Création de la ligne de données avec gestion des NaN
            data_rows.append({
                'id': id_counter,
                'total_cases': int(row['Confirmed']) if not pd.isna(row['Confirmed']) else 0,
                'total_deaths': int(row['Deaths']) if not pd.isna(row['Deaths']) else 0,
                'new_cases': 0,  # Simplifié pour l'exemple
                'new_deaths': 0,  # Simplifié pour l'exemple
                'id_location': location_id,
                'id_pandemie': pandemie_id,
                'id_calendar': calendar_id
            })
            
            id_counter += 1
    
    df_data = pd.DataFrame(data_rows)
    
    print(f"Table data préparée: {len(df_data)} lignes")
    return df_data

# Préparation des tables
df_calendar = prepare_calendar_table(transformed_data)
df_location = prepare_location_table(transformed_data)
df_pandemie = prepare_pandemie_table()
df_data = prepare_data_table(transformed_data, df_calendar, df_location)

# Stockage des tables dans un dictionnaire
tables = {
    'calendar': df_calendar,
    'location': df_location,
    'pandemie': df_pandemie,
    'data': df_data
}

## 5. Chargement des données dans Epiviz

In [ ]:
# Section pour la connexion à MySQL
import mysql.connector
from mysql.connector import Error

# Configuration de la base de données
db_config = {
    "host": "localhost",
    "port": 3306,
    "user": "root",
    "password": "",
    "database": "epiviz",
    "batch_size": 1000
}

def create_mysql_connection(config=db_config):
    """Crée une connexion à la base de données MySQL"""
    try:
        connection = mysql.connector.connect(
            host=config["host"],
            user=config["user"],
            password=config["password"],
            database=config["database"],
            port=config["port"]
        )
        if connection.is_connected():
            print(f"Connecté à MySQL, version {connection.get_server_info()}")
            return connection
    except Error as e:
        print(f"Erreur lors de la connexion à MySQL: {e}")
        return None

# Connexion à MySQL pour la visualisation
mysql_conn = create_mysql_connection()

if mysql_conn:
    # Requête pour les cas COVID par continent
    query_covid_by_continent = """
    SELECT c.date_value, l.continent, SUM(d.total_cases) as total_cases
    FROM data d
    JOIN calendar c ON d.id_calendar = c.id
    JOIN location l ON d.id_location = l.id
    WHERE d.id_pandemie = 1
    GROUP BY c.date_value, l.continent
    ORDER BY c.date_value
    """
    
    # Exécution de la requête
    cursor = mysql_conn.cursor()
    cursor.execute(query_covid_by_continent)
    rows = cursor.fetchall()
    
    # Conversion en DataFrame
    df_covid_by_continent = pd.DataFrame(rows, columns=['date_value', 'continent', 'total_cases'])
    df_covid_by_continent['date'] = pd.to_datetime(df_covid_by_continent['date_value'].astype(str), format='%Y%m%d')
    
    # Visualisation
    plt.figure(figsize=(14, 7))
    for continent in df_covid_by_continent['continent'].unique():
        data = df_covid_by_continent[df_covid_by_continent['continent'] == continent]
        plt.plot(data['date'], data['total_cases'], label=continent)
    
    plt.title('Évolution des cas COVID-19 par continent')
    plt.xlabel('Date')
    plt.ylabel('Nombre de cas')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Requête pour les top pays
    query_top_countries = """
    SELECT l.country, MAX(d.total_cases) as max_cases, p.type as pandemic_type
    FROM data d
    JOIN location l ON d.id_location = l.id
    JOIN pandemie p ON d.id_pandemie = p.id
    GROUP BY l.country, p.type
    ORDER BY max_cases DESC
    LIMIT 10
    """
    
    # Exécution de la requête
    cursor.execute(query_top_countries)
    rows = cursor.fetchall()
    
    # Conversion en DataFrame
    df_top_countries = pd.DataFrame(rows, columns=['country', 'max_cases', 'pandemic_type'])
    
    # Visualisation
    plt.figure(figsize=(12, 6))
    sns.barplot(x='country', y='max_cases', hue='pandemic_type', data=df_top_countries)
    plt.title('Top 10 pays par nombre de cas')
    plt.xlabel('Pays')
    plt.ylabel('Nombre de cas')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    mysql_conn.close()
else:
    print("Impossible de se connecter à MySQL pour la visualisation. Vérifiez vos paramètres de connexion.")